# Pandas Data Aggregation

`.groupby()`, `.agg()`

KPI Library: https://bernardmarr.com/kpi-library/

In [ ]:
import numpy as np
import pandas as pd

__As always we explore our dataset__

[Original dataset](https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=67663c0a55e16710VgnVCM1000001d4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default)

In [ ]:
%%time

parking = pd.read_csv('https://datos.madrid.es/egobfiles/MANUAL/300470/tiques_ser_1T_2023.zip')
#parking = pd.read_parquet()
#enero-mazro

In [ ]:
# Dimension tables 

distritos = pd.read_csv('./datasets/distritos.csv')
barrios = pd.read_csv('./datasets/barrios.csv')
#Necesitamos ponerle un sep = '\t' para que no quede cod_distrito\t\cod_barrio

In [ ]:
print(parking.info())
parking.head()

In [ ]:
print(distritos.info())
distritos.head()

In [ ]:
print(barrios.info())
barrios.head()

In [ ]:
parking.isnull().sum()
#Vemos los nulos que tenemos en el dataset

__Time span__

In [ ]:
%%time

parking['fecha_operacion'] = pd.to_datetime(parking['fecha_operacion'], format="%Y-%m-%d %H:%M:%S")
#Convertimos la fecha de una buena forma

In [ ]:
parking['fecha_operacion'].max() - parking['fecha_operacion'].min()

__Data Manipulation__

In [ ]:
%%time
#una vez codificadas las tablas realizamos un merge de parking con distritos. Agregar una columna adicional que tiene distritos

p_distritos = pd.merge(parking, distritos)
p_distritos.head()
#Debe haber 11 columnas, por lo tanto le hacemos un p_distrito.info()
#Debe haber como maximo las mismas filas que en el anterior,

In [ ]:
# Always check the results!!!
#un nuevo df que es un slicing de "[p_distritos['cod_distrito'].isin([5])]" 
#debe salir el distrito 5
just_checking = p_distritos[p_distritos['cod_distrito'].isin([5])]['distrito']
just_checking.unique()

---

## The `df.groupby()` object

In [ ]:
%%time
#Lo agrupamos por distritos con group.by
groupby_object = p_distritos.groupby(['distrito'])
groupby_object

## The `.agg()` method

In [ ]:
%%time
#Necesito el tipo de agregación(suma) y lo que voy a agregar (minutos e importe)
total_distritos = groupby_object[['minutos_tique',
                                  'importe_tique']]\
                  .sum()#.sort_values(by='importe_tique',
                                     #ascending=False).reset_index() si pones index, el distrito apareceria como columna, no como index
total_distritos

---

In [ ]:
# Another aggregation

media_distritos = p_distritos.groupby(['distrito'])[['minutos_tique',
                                                     'importe_tique']]\
                  .mean()#.sort_values(by='importe_tique',
                                      #ascending=False).reset_index()
media_distritos

In [ ]:
#Multiple agg functions for all columns

stats_distritos = p_distritos.groupby(['distrito'])[['minutos_tique',
                                                     'importe_tique']]\
                  .agg(['min', 'max', 'mean', 'median', 'std'])\
                  .sort_values(by=('importe_tique',
                                   'median'), ascending=False)
stats_distritos
#Lo que saldrá arriba es un multi-index
#saca el 'min', etc... de los minutos y del importe

In [ ]:
stats_distritos.index

In [ ]:
stats_distritos.columns

In [ ]:
# Multiple agg functions for different columns

multistats_distritos = p_distritos.groupby(['distrito'])[['minutos_tique',
                                                          'importe_tique']]\
                       .agg(mean_importe=('importe_tique', 'mean'),
                            std_importe=('importe_tique', 'std'),
                            median_minutos=('minutos_tique', 'median')).sort_values(by='median_minutos',
                                                                                    ascending=False)
multistats_distritos
#definir un nombre(mean_importe), es decir agregaciones especificas para columnas especificas


In [ ]:
# Numpy agg function (e.g.: https://numpy.org/doc/stable/reference/generated/numpy.ptp.html)

numpy_distritos = p_distritos.groupby(['distrito'])[['importe_tique']]\
                  .agg(np.ptp).sort_values(by='importe_tique', ascending=False)
numpy_distritos
#el rango maximo del importe de cada ticket en cada distrito

In [ ]:
#Custom agg function

def custom(col):
    return f'{round((np.sum(col)*100)/ minutos, 2)} centimos/hora'
#

In [ ]:
minutos = p_distritos['minutos_tique'].sum() / 60

custom_distritos = p_distritos.groupby(['distrito'])[['importe_tique']]\
                   .agg(custom).sort_values(by='importe_tique', ascending=False)
custom_distritos

---

### Now your turn to practice performing the same excercise by _'Barrio'_...

![Image](./img/etl_pandas_agg_01.jpg)